In [1]:
import numpy as np

# US Exercise

We are going to use the data calibrated to the US economy to evaluate the impact of technology and labor supply shocks. 

In [ ]:
data_dir = 'data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_monthly= pd.read_csv(data_dir + 'labor_market_monthly.csv')
dfLabor_market_monthly.date = pd.to_datetime(dfLabor_market_monthly.date)
dfLabor_market_monthly = dfLabor_market_monthly.sort_values(by=['date', 'BEA_sector'])
dfLabor_market_monthly = dfLabor_market_monthly.dropna(axis=0)

# reformatting parameters
A = np.array(dfA.iloc[:, 1:], dtype='float64')
φ = np.array(dfParam.φ)
λ = np.array(dfParam.λ)
λ_alt = np.array(dfParam.λ_alt)

α = np.array(dfParam.α)
θ = np.array(dfParam.θ)
θ_alt = np.array(dfParam.θ)
γ = np.array(dfParam.γ)

In [2]:
# Size of network
J = 4 

# Sampling input-output matrix entries 
μ = np.zeros(J+1)
Σ = np.eye(J+1)
z_draws = np.random.multivariate_normal(μ, Σ, J+1)
elasticity_draws = np.exp(z_draws)/np.sum(np.exp(z_draws),1).reshape((J+1,1))

# Elasticities
epsD = elasticity_draws[0,:-1]/np.sum(elasticity_draws[0,:-1])
Omega = elasticity_draws[1:,:-1]
Psi = np.linalg.inv(np.eye(J)-Omega)
epsN = np.diag(elasticity_draws[1:,-1])

# Drawing elasticity of matching function wrt to U
ν = np.random.uniform(size=J)
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) + curlyQ

Finally, we need recruiter producer ratios $\tau_j(\theta_j)$ in each sector. Landais, Michaillat, and Saez (2018) find that the share of recruiters in the US workforce averages around 2.3 percent. I sample recruiter producer ratios uniformly on the interval $[0,0.046]$ to roughly match this fact.

In [3]:
tau = np.diag(np.random.uniform(low=0,high=0.046,size=J))
tau

array([[0.01857811, 0.        , 0.        , 0.        ],
       [0.        , 0.02767437, 0.        , 0.        ],
       [0.        , 0.        , 0.00146646, 0.        ],
       [0.        , 0.        , 0.        , 0.04083065]])

## Defining shocks
We are interested in the response of sector level and aggregate output and employment to technology shocks $d\log\bm{A}$ and labor force shocks $d\log \bm{H}$. The code below defines the shocks we feed into the model.

In [4]:
# Technology shocks
dlog_A = 0.01*np.ones(J)
dlog_H = 0.01*np.zeros(J)

## Defining how wages adjust
Since there are mutual gains from trade once an unemployed worker and a firm meet, wages are not pinned down uniquely in models featuring search and matching frictions in the labor market. We must therefore impose a wage schedule, an assumption about how wages change in response to fundamentals, in order to close the model. 

In particular, we need to specify the elasticity of wages to technology shocks and labor force shocks in each sector, $\left\{\left\{\varepsilon^{w_j}_{A_{ji}},\varepsilon^{w_j}_{H_{ji}}\right\}_{i=1}^{J}\right\}_{j=1}^{J}$. For instance, a simple, but unrealistic, assumption is that wages respond positively to own sector productivity and negatively to own sector labor force, but do not respond to changes in other sectors. In general, we express changes in wages as a function of changes in productivity and the labor force. 
\begin{align}
    d\log \bm{w} &= \bm{\Lambda_{A}} d\log \bm{A} + \bm{\Lambda_{H}} d\log \bm{H} \tag{1}
\end{align}
Where $\bm{\Lambda_{A}}$ contains wage elasticities to productivity changes and $\bm{\Lambda_{H}}$ contains wage elasticities to labor force changes.

In [5]:
# Wage elasticities
epsW_A = np.diag(np.random.uniform(low=0,high=2,size=J))
epsW_H = np.diag(np.random.uniform(low=-2,high=0,size=J))
epsW_A

array([[1.96654215, 0.        , 0.        , 0.        ],
       [0.        , 1.45220152, 0.        , 0.        ],
       [0.        , 0.        , 1.84644965, 0.        ],
       [0.        , 0.        , 0.        , 1.14286024]])

In [6]:
# Calculating the log change in wages
def WageFunc(dlog_A, dlog_H, epsW_A, epsW_H):
    dlog_w = epsW_A @ dlog_A + epsW_H @ dlog_H
    return dlog_w

In [7]:
#How wages change
dlog_w = WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)
dlog_w

array([0.01966542, 0.01452202, 0.0184645 , 0.0114286 ])

## Tightness propagation
We with wage changes in hand, we solve for first order changes in tightness in terms of $d\log\bm{A}$, $d\log\bm{H}$, and $d\log\bm{w}$. The general formula for changes in tightness, treating sector 1 prices as the numeraire, is
\begin{align}
    d \log \bm{\theta} &=\left(\text{diag}\left(\bm{\mathcal{F}}\right)-\bm{\Xi_{\theta}}\right)^{-1}\left(\left(\bm{I} - \bm{\Psi} \text{diag}\left(\bm{\varepsilon^f_N}\right)\right) \left(d\log \bm{\varepsilon^f_N} + d\log \bm{\lambda} - d\log \bm{H}\right) + \bm{\Xi_{A}} d\log \bm{A}\right) \nonumber \\
    &- \left(\text{diag}\left(\bm{\mathcal{F}}\right)-\bm{\Xi_{\theta}}\right)^{-1} \left(\bm{I}-\bm{\Xi}_{w}\right)d\log \bm{w} \tag{2}
\end{align}
Where 
\begin{align*}
    \bm{\Xi_{A}} &= \begin{bmatrix}
        \Psi_1 \\
        \bm{0} \\
        \vdots \\
        \bm{0}
    \end{bmatrix}, \, \bm{\Xi_\theta} = \begin{bmatrix}
        \Psi_1 \text{diag}(\varepsilon^{f}_{N})\left(\text{diag}(\varepsilon^{\mathcal{F}}_{\theta}) + \text{diag}(\tau)\text{diag}(\varepsilon^{\mathcal{Q}}_{\theta})\right) \\
        \Psi_2 \text{diag}(\varepsilon^{f}_{N}) \text{diag}(\varepsilon^{\mathcal{F}}_{\theta}) \\
        \vdots \\
        \Psi_J \text{diag}(\varepsilon^{f}_{N}) \text{diag}(\varepsilon^{\mathcal{F}}_{\theta})
    \end{bmatrix}, \bm{\Xi_{w}} = \begin{bmatrix}
        \bm{0} \\
        \Psi_2 \text{diag}(\varepsilon^{f}_{N}) \\
        \vdots\\
        \Psi_J \text{diag}(\varepsilon^{f}_{N})
    \end{bmatrix}
\end{align*}

In [8]:
def ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau, num=0):
    
    # Creating matrices
    Xi_a = np.zeros_like(Psi)
    Xi_a[num,:] = Psi[num,:]

    Xi_theta = Psi @ epsN @ curlyF
    Xi_theta[num,:] = Xi_theta[num,:] + Psi[num,:] @ epsN @ tau @ curlyQ

    Xi_w = Psi @ epsN 
    Xi_w[num,:] = 0

    I = np.eye(Psi.shape[0])

    # Contribution of different components
    Cw = np.linalg.inv(curlyF - Xi_theta) @ (I - Xi_w)
    Ca = np.linalg.inv(curlyF - Xi_theta) @ Xi_a 
    Ch = np.linalg.inv(curlyF - Xi_theta) @ (I - Psi @ epsN)

    # Change in tightness
    dlog_theta = Ch @ (dlog_epsN + dlog_lam - dlog_H) + Ca @ dlog_A - Cw @ dlog_w

    return dlog_theta

Assuming Cobb-Douglas production implies $d\log\bm{\varepsilon^f_N} = d\log \bm{\lambda} = 0$.

In [9]:
dlog_epsN = np.zeros_like(dlog_A)
dlog_lam = np.zeros_like(dlog_A)
dlog_theta = ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau)
dlog_theta

array([ 2.92851566,  4.33216931,  2.62834351, 13.31995885])

## Price and output propagation
With changes in tightness in hand, we can now work out how prices and sectoral production changes in response to technology and labor force shocks. Price changes are given by 
\begin{align}
    d \log \bm{p} &=\bm{\Psi} \left(
        \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log\bm{w} -\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)d\log\bm{\theta} - d\log \bm{A} \right) \tag{3}
\end{align}
Since we have assumed that sector one is the numerair, we need to check that price changes are consistent with this assumption. Unfortunately, below it looks like they are not, which means we may have to think a bit harder about the pricing equation. 

In [10]:
def PriceFunc(dlog_A, dlog_w, Psi, curlyQ, epsN, tau):
    # Contributions of different components
    Cw = Psi @ epsN 
    Ct = Psi @ epsN @ tau @ curlyQ
    Ca = Psi
    # Price changes
    dlog_p = Cw @ dlog_w - Ct @ dlog_theta - Ca @dlog_A
    return dlog_p

In [11]:
dlog_p = PriceFunc(dlog_A, dlog_w, Psi, curlyQ, epsN, tau)
dlog_p

array([-1.28425048e-13, -2.48671686e-03,  4.71561775e-03,  3.72007269e-02])

And output changes are given by
\begin{align}
    d\log \bm{y} &= \bm{\Psi}\left(d\log\bm{A} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\left(\text{diag}\left(\bm{\mathcal{F}}\right)+\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)\right)d\log \bm{\theta} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log\bm{H}\right) \nonumber\\
    &-\bm{\Psi} \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log \bm{\varepsilon^{f}_{N}} + \left(\bm{I} - \bm{\Psi}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right) d\log \bm{\lambda} \tag{4}
\end{align}


In [12]:
def OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_epsN, dlog_lam, Psi, curlyQ, curlyF, epsN, tau):
    # Contributions of different coponents
    Ca = Psi
    Ct = Psi @ epsN @ (curlyF + tau @ curlyQ) 
    Ch = Psi @ epsN 
    I = np.eye(Psi.shape[0])

    # Changes in output
    dlog_y = Ca @ dlog_A + Ct @ dlog_theta + Ch @ (dlog_H-dlog_epsN) + (I - Ch) @ dlog_lam 

    return dlog_y

In [13]:
dlog_y = OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_epsN, dlog_lam, Psi, curlyQ, curlyF, epsN, tau)
dlog_y

array([2.45693237, 2.45941908, 2.45221675, 2.41973164])

In [14]:
# Change in nominal GDP
dlog_p + dlog_y

array([2.45693237, 2.45693237, 2.45693237, 2.45693237])

We can now check consistency. We should get the same change in labor by either calculating the change in labor supply
\begin{align}
    d\log \bm{L^s} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}
Or the change in labor demand
\begin{align}
    d\log \bm{L^d} = d\log \bm{\varepsilon^{f}_N} + d\log \bm{p} + d\log\bm{y} - d\log\bm{w} \tag{6}
\end{align}

In [15]:
def LaborSupply(dlog_H,dlog_theta,curlyF):
    dlog_Ls = curlyF @ dlog_theta + dlog_H
    return dlog_Ls
    
def LaborDemand(dlog_w, dlog_p, dlog_y, dlog_epsN):
    dlog_Ld = dlog_epsN + dlog_p + dlog_y - dlog_w
    return dlog_Ld

Consistency requires
\begin{align*}
    d\log \bm{L^d} - d\log\bm{L^s} = 0
\end{align*}

In [16]:
LaborDemand(dlog_w, dlog_p, dlog_y, dlog_epsN) - LaborSupply(dlog_H,dlog_theta,curlyF)

array([-1.28341782e-13,  0.00000000e+00,  4.44089210e-16, -4.44089210e-16])

## Unemployment
We can use changes in tightness and the labor force to calculate changes in employment. 
\begin{align}
    d\log \bm{L} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}

In [17]:
dlog_L = LaborSupply(dlog_H,dlog_theta,curlyF)
dlog_L

array([2.43726695, 2.44241035, 2.43846787, 2.44550376])

Unemployment in sector $j$ is $U_j = H_j - L_j$. The unemployment rate in sector $j$ is $u_j = \frac{H_j-L_j}{H_j}$.


## Aggregation